# Hadamard Test
---


The Hadamard test is a quantum circuit that allows estimation of the real and imaginary parts of the expected value of a unitary operator. It is a fundamental subroutine used in many quantum algorithms. The test works by applying a controlled-unitary operation between an auxiliary qubit and the system of interest, with the measurement statistics of the auxiliary qubit encoding information about the unitary's expectation value. For the original paper, see [1]. For a more modern treatment, see [2].

In this notebook, we explore the Braket implementation of the Hadamard test for estimating the real and imaginary parts of the expected value of a unitary operator.


## References

[1] R. Cleve, A. Ekert, C. Macchiavello, and M. Mosca (1998). Quantum algorithms revisited. [arXiv:quant-ph/9708016](https://arxiv.org/abs/quant-ph/9708016).

[2] Nielsen, Michael A., Chuang, Isaac L. (2010). Quantum Computation and Quantum Information (2nd ed.). Cambridge: Cambridge University Press.

## Description

Given a unitary operator $U$ and qubits $|0\rangle|\psi\rangle$, the Hadamard test is used to estimate the expected value of $U$ on $|\psi\rangle$, i.e. $\langle \psi | U |\psi \rangle$. We will walk through the algorithm step by step. First, we apply a Hadamard gate to $|0\rangle$, putting the system into the state,
 
$$
\dfrac{|0\rangle + |1\rangle}{\sqrt{2}}|\psi\rangle.
$$ 
 
We then apply the unitary operator $U$ on $|\psi\rangle$, controlled on the first qubit, to obtain the state, 
$$
\dfrac{|0\rangle |\psi\rangle + |1\rangle U |\psi\rangle}{\sqrt{2}} .
$$ 
Then, a Hadamard gate is applied to the first qubit once more, giving the state,
$$
\dfrac{|0\rangle(|\psi\rangle + U |\psi\rangle) + |1\rangle(|\psi\rangle - U |\psi\rangle)}{2}.
$$

And this completes the Hadamard test. Now, we get the probability of measuring the first qubit in the state $|0\rangle$ as
$$
\begin{align*}
P_0 &= \bigg\lVert\dfrac{|\psi\rangle + U|\psi\rangle}{2}\bigg\rVert^2 \\
    &= \dfrac{\langle \psi | \psi \rangle + \langle \psi | U |\psi \rangle + \langle \psi | U^\dagger |\psi \rangle + \langle \psi | U^\dagger U |\psi \rangle}{4} \\
    &= \dfrac{1 + 2\langle \psi | U |\psi \rangle + 1}{4}\\
    &= \dfrac{1 + Re\langle \psi | U |\psi \rangle}{2},
\end{align*}
$$
allowing us to estimate the expected value of the real part as $ Re(\langle \psi | U |\psi \rangle) = 2 P_0 - 1$. The algorithm can be modified to estimate the imaginary part by applying a phase shift to the first qubit after the first Hadamard gate, the proof of which is left as an exercise or inspiration for further reading.


## Run on a local simulator

Braket provides an implementation of the Hadamard test, `hadamard_test_circuit`, for estimating the expected values of both the real and imaginary parts of a given unitary operator, `U`.


In [5]:
from notebook_plotting import plot_bitstrings_formatted
import numpy as np

%matplotlib inline

from braket.aws import AwsDevice
from braket.circuits import Circuit, Qubit
from braket.devices import LocalSimulator
from braket.tracking import Tracker

from braket.experimental.algorithms.hadamard_test.hadamard_test import hadamard_test_circuit

tracker = Tracker().start()  # to track Braket costs

For our controlled unitary operator, we will use the Pauli-Z gate, which is represented by the matrix,
$$
\begin{bmatrix}
1 & 0 \\
0 & -1
\end{bmatrix}
$$
Though Braket has built-in support for the Pauli-Z gate, we will use the `unitary` method to apply the gate to the qubit. Consider playing around with different matrices to test your understanding of the algorithm.

In [6]:
pauli_z = np.array([[1, 0], [0, -1]])
controlled_unitary = Circuit().unitary([0], pauli_z, "U")
ancilla = Qubit(0)
print(controlled_unitary)

T  : │  0  │
      ┌───┐ 
q0 : ─┤ U ├─
      └───┘ 
T  : │  0  │


### Real Part Estimation

In [8]:
ht_circuit = hadamard_test_circuit(ancilla, controlled_unitary)
print(ht_circuit)

T  : │  0  │  1  │  2  │
      ┌───┐       ┌───┐ 
q0 : ─┤ H ├───●───┤ H ├─
      └───┘   │   └───┘ 
            ┌─┴─┐       
q1 : ───────┤ U ├───────
            └───┘       
T  : │  0  │  1  │  2  │


By default, Braket will measure all qubits. Since the Hadamard test only requires measuring the ancilla qubits, we will explicitly add a measurement instruction to the circuit to avoid unnecessary measurements.

In [9]:
ht_circuit.measure(0)
print(ht_circuit)

T  : │  0  │  1  │  2  │  3  │
      ┌───┐       ┌───┐ ┌───┐ 
q0 : ─┤ H ├───●───┤ H ├─┤ M ├─
      └───┘   │   └───┘ └───┘ 
            ┌─┴─┐             
q1 : ───────┤ U ├─────────────
            └───┘             
T  : │  0  │  1  │  2  │  3  │


Now, we can run the circuit on a local simulator. Take note of the small bit of algebra we need to extract the real part, which can also be seen in the description of the algorithm above.

In [10]:
device = LocalSimulator()
task = device.run(ht_circuit, shots=1000)

probs  = task.result().measurement_probabilities
p_zero = probs.get('0', 0)

real_part = 2 * p_zero - 1

print(real_part)

1.0


### Imaginary Part Estimation

Braket's `hadamard_test_circuit` supports the real part estimation by default, but we can also estimate the imaginary part by setting the `component` argument to `'imaginary'`. We will use the same controlled unitary operator as before.

In [12]:
ht_circuit = hadamard_test_circuit(Qubit(0), controlled_unitary, component='imaginary')
ht_circuit.measure(0)
print(ht_circuit)

T  : │  0  │  1  │  2  │  3  │  4  │
      ┌───┐ ┌───┐       ┌───┐ ┌───┐ 
q0 : ─┤ H ├─┤ S ├───●───┤ H ├─┤ M ├─
      └───┘ └───┘   │   └───┘ └───┘ 
                  ┌─┴─┐             
q1 : ─────────────┤ U ├─────────────
                  └───┘             
T  : │  0  │  1  │  2  │  3  │  4  │


The only difference in the circuit is the addition of a phase shift gate to the first qubit, which allows us to estimate the expected value of the imaginary part.

In [13]:
device = LocalSimulator()
task = device.run(ht_circuit, shots=1000)

probs  = task.result().measurement_probabilities
p_zero = probs.get('0', 0)

imaginary_part = 2 * p_zero - 1

print(imaginary_part)

-0.01200000000000001


## Run on a QPU 

To run the Hadamard test on a QPU, we replace the LocalSimulator with an AwsDevice. The cost to run this experiment is $0.3 per task and $0.00145 per shot on the IQM Garnet device, that totals $1.75 USD. Because Garnet does not support arbitrary unitary operators, we will directly apply the Pauli-Z gate in this example.

In [14]:
# controlled_unitary = Circuit().z(0)
# ht_circuit = hadamard_test_circuit(controlled_unitary)
# ht_circuit.measure(0)
# print(ht_circuit)

# device = AwsDevice("arn:aws:braket:eu-north-1::device/qpu/iqm/Garnet")
# task = device.run(ht_circuit, shots=1000)

# counts = task.result().measurement_counts
# p_zero = counts.get('0', 0) / 1000

# real_part = 2 * p_zero - 1

# print(real_part)

In [15]:
print("Task Summary")
print(f"{tracker.quantum_tasks_statistics()} \n")
print(
    f"Estimated cost to run this example: {tracker.qpu_tasks_cost() + tracker.simulator_tasks_cost():.2f} USD"
)

Task Summary
{} 

Estimated cost to run this example: 0.00 USD


Note: Charges shown are estimates based on your Amazon Braket simulator and quantum processing unit (QPU) task usage. Estimated charges shown may differ from your actual charges. Estimated charges do not factor in any discounts or credits, and you may experience additional charges based on your use of other services such as Amazon Elastic Compute Cloud (Amazon EC2).